# Introduction to Modeling with Python

### Simple Python Example

In [1]:
import gurobipy as gp
from gurobipy import GRB as grb

In [4]:
m = gp.Model("mip1")
# Decision variables
x = m.addVar(vtype=grb.BINARY, name="x")
y = m.addVar(vtype=grb.BINARY, name="y")
z = m.addVar(vtype=grb.BINARY, name="z")

# Objective function
m.setObjective(x + y + 2*z, grb.MAXIMIZE)
# Add constraints
m.addConstr(x + 2*y + 3*z <= 4, name="c1")
m.addConstr(x + y >= 1, name="c2")

m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xb2adf8c4
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


#### Gurobi Tuplelist

In [6]:
arcs = gp.tuplelist([("CHI", "NYC"), ("CHI", "ATL"), ("ATL", "MIA"), ("ATL", "NYC")])

In [8]:
print(arcs.select("CHI", "*")) # select() method finds matching subsets

<gurobi.tuplelist (2 tuples, 2 values each):
 ( CHI , NYC )
 ( CHI , ATL )
>


#### Gurobi Multidict

In [11]:
cities, supply, demand = gp.multidict({"ATL": [100, 20],
                                       "CHI": [150, 50],
                                       "NYC": [20, 300],
                                       "MIA": [10,  20]})

In [10]:
print(cities)

['ATL', 'CHI', 'NYC', 'MIA']


#### Good vs Bad practices

# Advanced Modeling 
by Matthias Miltenberger

#### Gurobi Tuplelist (Cont.)

In [16]:
from gurobipy import * # is importing everything is a good practice ?

In [59]:
m = gp.Model("mip2")

In [60]:
cities = ['A', 'B', 'C', 'D']
routes = tuplelist([('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D')])

In [61]:
[print(routes.select(c, '*')) for c in cities]

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>
<gurobi.tuplelist (2 tuples, 2 values each):
 ( B , C )
 ( B , D )
>
<gurobi.tuplelist (1 tuples, 2 values each):
 ( C , D )
>
[]


[None, None, None, None]

In [19]:
for c in cities:
    print(routes.select(c, '*'))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>
<gurobi.tuplelist (2 tuples, 2 values each):
 ( B , C )
 ( B , D )
>
<gurobi.tuplelist (1 tuples, 2 values each):
 ( C , D )
>
[]


#### Indexed variables: Model.addVars()

###### Using integers

In [20]:
x = m.addVars(2, 3, name="x")

In [39]:
for v in x:
    print(v)

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)


###### Using lists of scalars

In [27]:
y = m.addVars(cities, cities, name="y")

In [36]:
for v in y:
    print(v)

('A', 'A')
('A', 'B')
('A', 'C')
('A', 'D')
('B', 'A')
('B', 'B')
('B', 'C')
('B', 'D')
('C', 'A')
('C', 'B')
('C', 'C')
('C', 'D')
('D', 'A')
('D', 'B')
('D', 'C')
('D', 'D')


###### Using a tuplelist

In [40]:
z = m.addVars(routes, name="z")
for v in z:
    print(v)

('A', 'B')
('A', 'C')
('B', 'C')
('B', 'D')
('C', 'D')


###### Using a generator expression

In [45]:
w = m.addVars((i for i in range(5) if i!=2), name="w")
for v in w:
    print(v)

0
1
3
4


#### Indexed constraints: Model.addConsts()

In [62]:
x = m.addVars(routes, name="x")
y = m.addVars(routes, name="y")
m.addConstrs((x[i,j] + y[i, j] <= 2 for i,j in routes), name="capacity")

{('A', 'B'): <gurobi.Constr *Awaiting Model Update*>,
 ('A', 'C'): <gurobi.Constr *Awaiting Model Update*>,
 ('B', 'C'): <gurobi.Constr *Awaiting Model Update*>,
 ('B', 'D'): <gurobi.Constr *Awaiting Model Update*>,
 ('C', 'D'): <gurobi.Constr *Awaiting Model Update*>}

###### Aggregate sum: using tupledict.sum()

In [63]:
x = m.addVars(3,4, vtype=grb.BINARY, name="x")
m.addConstrs(x.sum(i, '*') <= 1 for i in range(3))  # good practice
m.update()
print(m.display())

Minimize
   <gurobi.LinExpr: 0.0>
Subject To
   capacity[A,B] : <gurobi.LinExpr: x[A,B] + y[A,B]> <= 2.0
   capacity[A,C] : <gurobi.LinExpr: x[A,C] + y[A,C]> <= 2.0
   capacity[B,C] : <gurobi.LinExpr: x[B,C] + y[B,C]> <= 2.0
   capacity[B,D] : <gurobi.LinExpr: x[B,D] + y[B,D]> <= 2.0
   capacity[C,D] : <gurobi.LinExpr: x[C,D] + y[C,D]> <= 2.0
   R5 : <gurobi.LinExpr: x[0,0] + x[0,1] + x[0,2] + x[0,3]> <= 1.0
   R6 : <gurobi.LinExpr: x[1,0] + x[1,1] + x[1,2] + x[1,3]> <= 1.0
   R7 : <gurobi.LinExpr: x[2,0] + x[2,1] + x[2,2] + x[2,3]> <= 1.0
Binaries
   ['x[0,0]', 'x[0,1]', 'x[0,2]', 'x[0,3]', 'x[1,0]', 'x[1,1]', 'x[1,2]', 'x[1,3]', 'x[2,0]', 'x[2,1]', 'x[2,2]', 'x[2,3]']
None


###### Aggregate sum: using quicksum()
whenever you want to use python sum, or sum() over variables use quicksum()

In [65]:
obj = quicksum(cost[i,j]*x[i,j] for i,j in routes) # cost is not defined beforhand

NameError: name 'cost' is not defined

#### Gurobi Tupledict 